In [450]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
import lightgbm as lgb
from lightgbm import LGBMClassifier
import scipy.stats as stats
from sklearn.feature_extraction import text
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

from tqdm import tqdm, tqdm_notebook
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
%matplotlib inline

# 1. 对原始数据进行孤立森林操作（新增一列）

In [266]:
train1 = pd.read_csv('data/train_xy.csv')
train2 = pd.read_csv('data/train_x.csv')
test = pd.read_csv('data/test_all.csv')
# 把train和test先拼接起来
train2['y'] = -1
test['y'] = -2
org_data = pd.concat([train1, train2, test],axis=0,sort=False)
del train1, train2, test

In [272]:
d_ = {'group_1':1,'group_2':2,'group_3':3}
org_data['cust_group'] = org_data['cust_group'].apply(lambda x : d_.get(x))

In [270]:
isof = IsolationForest(n_jobs=40)
isof.fit(org_data.drop(['cust_id','y'],axis=1))
org_data['isof'] = isof.predict(org_data.drop(['cust_id','y'],axis=1))

# 2. 对原始数据进行KMeans操作（新增一列）

In [ ]:
km = KMeans(n_clusters=10)
tmp = km.fit_transform(org_data.drop(['cust_id','y'],axis=1))
org_data['km_feature'] = tmp.argmax(axis=1)
del tmp

# 2. 加载1_baseline中生成的data1

In [286]:
data1 = pd.read_hdf('data/data1.h5',key='data')
data1['isof'] = org_data['isof']
data1['km_feature'] = org_data['km_feature']

# 3. 对top40数值特征进行poly衍生

In [ ]:
# top40特征用lgb跑一些model然后拿到重要的前几个features

In [103]:
rf = RowFeature()

In [247]:
tmp1 = data1[f_40_num].copy()

In [288]:
tmp1.head()

,x_1,x_2,x_40,x_41,x_42,x_43,x_44,x_45,x_46,x_47,...,x_81,x_93,x_95,f47_78,f39_46,f82_89,f38_1520_2530,f914_2124_3138,cate_na_count,num_na_count
0,0.354167,0.604988,9.0,10.0,7.0,9.0,7.0,6.0,2.0,8.0,...,0.969147,0.208333,0.436170,113.0,56.0,8.0,13.007526,12.991645,38.0,38.0
1,0.125000,0.012058,8.0,7.0,7.0,3.0,10.0,6.0,3.0,7.0,...,0.905626,0.270833,0.297872,158.0,51.0,10.0,13.007526,12.991645,37.0,38.0
2,0.333333,0.565979,1.0,3.0,6.0,5.0,7.0,3.0,2.0,8.0,...,0.867514,0.185821,0.372340,87.0,29.0,19.0,8.000000,8.000000,37.0,3.0
3,0.208333,0.316209,2.0,3.0,7.0,9.0,7.0,9.0,2.0,7.0,...,0.907441,0.010417,0.053191,97.0,41.0,47.0,6.000000,20.000000,37.0,2.0
4,0.208333,0.008061,9.0,11.0,10.0,9.0,10.0,7.0,3.0,11.0,...,0.865699,0.052083,0.095745,131.0,61.0,8.0,13.007526,6.000000,37.0,20.0


In [289]:
tmp2 = rf.get_feats_row_poly(tmp1, degree=2)
tmp2.drop(f_40_num,axis=1,inplace=True)

In [292]:
fs = FeatureSelector(tmp2)
fs.identify_single_unique()
fs.identify_collinear(correlation_threshold=0.9)

No labels provided. Feature importance based methods are not available.
0 features with a single unique value.

Compute Corr Matrix ...


correlated features: 100%|██████████| 481/481 [00:02<00:00, 235.82it/s]

481 features with a correlation magnitude greater than 0.90.



In [293]:
tmp2.drop(fs.ops['single_unique'],axis=1,inplace=True)
tmp2.drop(fs.ops['collinear'],axis=1,inplace=True)

#### 对于值特别大的列进行归一化

In [303]:
for i in tmp2.columns:
    if tmp2[i].max() > 100:
        mms = MinMaxScaler(feature_range=(0,100))
        tmp2[i] = mms.fit_transform(tmp2[[i]]).ravel()

In [307]:
tmp2.head()

,x_1^2,x_1 x_2,x_1 x_40,x_1 x_41,x_1 x_42,x_1 x_43,x_1 x_44,x_1 x_45,x_1 x_46,x_1 x_48,...,x_95 f82_89,x_95 f38_1520_2530,x_95 f914_2124_3138,x_95 num_na_count,f82_89 f38_1520_2530,f82_89 f914_2124_3138,f82_89 num_na_count,f38_1520_2530^2,f914_2124_3138^2,cate_na_count^2
0,0.125434,0.214267,3.187500,3.541667,2.479167,3.187500,2.479167,2.125000,0.708333,3.541667,...,3.489362,5.673495,1.955783,16.574468,5.829703,3.074036,7.219512,1.082853,0.096139,24.705882
1,0.015625,0.001507,1.000000,0.875000,0.875000,0.375000,1.250000,0.750000,0.375000,0.750000,...,2.978723,3.874582,1.335657,11.319149,7.287129,3.842545,9.073171,1.082853,0.096139,20.869565
2,0.111111,0.188660,0.333333,1.000000,2.000000,1.666667,2.333333,1.000000,0.666667,1.333333,...,7.074468,2.978723,1.028089,1.117021,8.515406,4.495711,1.195122,0.409600,0.036455,20.869565
3,0.043403,0.065877,0.416667,0.625000,1.458333,1.875000,1.458333,1.875000,0.416667,1.250000,...,2.500000,0.319149,0.367175,0.106383,15.798319,27.802425,2.097561,0.230400,0.227841,20.869565
4,0.043403,0.001679,1.875000,2.291667,2.083333,1.875000,2.083333,1.458333,0.625000,0.833333,...,0.765957,1.245401,0.198274,1.914894,5.829703,1.419698,3.707317,1.082853,0.020506,20.869565


In [383]:
# 先跑一下单poly的结果看下线上成绩
tmp3 = pd.DataFrame(np.column_stack((data1.iloc[:,:135].values, tmp2.values)),columns=list(data1.iloc[:,:135].columns)+\
                    list(tmp2.columns))

# 看下poly和组合特征结合看下线上成绩 0.74
tmp3 = pd.DataFrame(np.column_stack((data1.values, tmp2.values)),columns=list(data1.columns)+\
                    list(tmp2.columns))

In [377]:
fs = FeatureSelector(tmp3.drop(['cust_id','y'], axis=1).copy())

No labels provided. Feature importance based methods are not available.


In [378]:
fs.identify_collinear(correlation_threshold=0.95)

Compute Corr Matrix ...


correlated features: 100%|██████████| 123/123 [00:00<00:00, 265.50it/s]

123 features with a correlation magnitude greater than 0.95.



In [379]:
tmp3.drop(fs.ops['collinear'],axis=1,inplace=True)

# 4. 跑模型

In [1233]:
params = {
    'boosting': 'gbdt', # 'rf', 'dart', 'goss'
    'application': 'binary', # 'application': 'multiclass', 'num_class': 3, # multiclass=softmax, multiclassova=ova  One-vs-All
    'learning_rate': 0.01,
    'max_depth': -1,
    'num_leaves': 39, # 根据具体问题调整
    
    'max_bin':255,
    'metric_freq':10,
    
    'min_split_gain': 0,
    'min_child_weight': 40,

    'bagging_fraction': 0.8,
    'feature_fraction': 0.8,
    'bagging_freq': 5,
    'min_data_in_leaf': 1000,
    'min_sum_hessian_in_leaf': 5.0,
    'lambda_l1': 0,
    'lambda_l2': 1,

    'scale_pos_weight': 1,
    'metric': 'auc',
    'num_threads': 40,
}

In [1234]:
X = tmp3[tmp3.y>=0].drop(['cust_id','y'],axis=1)
y = tmp3[tmp3.y>=0].y
lgb_data = lgb.Dataset(X, y)
res = lgb.cv(
    params,
    lgb_data,
#     categorical_feature = cate_features2,
    num_boost_round=2000,
    nfold=5,
    stratified=False,
    metrics=None,
    early_stopping_rounds=150,
    verbose_eval=50,
    show_stdv=True,
    seed=0
)

[50]	cv_agg's auc: 0.793967 + 0.014468
[100]	cv_agg's auc: 0.801608 + 0.013705
[150]	cv_agg's auc: 0.807529 + 0.0150191
[200]	cv_agg's auc: 0.80997 + 0.0160065
[250]	cv_agg's auc: 0.811707 + 0.0166004
[300]	cv_agg's auc: 0.812631 + 0.0175325
[350]	cv_agg's auc: 0.813918 + 0.0174352
[400]	cv_agg's auc: 0.814753 + 0.017893
[450]	cv_agg's auc: 0.815071 + 0.0178146
[500]	cv_agg's auc: 0.815713 + 0.0181036
[550]	cv_agg's auc: 0.81582 + 0.017677
[600]	cv_agg's auc: 0.815977 + 0.0172478
[650]	cv_agg's auc: 0.816012 + 0.016816
[700]	cv_agg's auc: 0.815893 + 0.0162897


In [1235]:
iter_best = len(res['auc-mean'])
print(iter_best)

596


In [1236]:
res_train = lgb.train(
    params,
    lgb_data,
#     categorical_feature = cate_features2,
    num_boost_round=iter_best
)

In [1237]:
pred = res_train.predict(tmp3[tmp3.y==-2].drop(['cust_id','y'],axis=1))

In [1061]:
result = tmp3[tmp3.y==-2][['cust_id']]
result['pred_prob'] = pred
result['cust_id'] = result['cust_id'].astype(int)
result.to_csv('output/baseline10.csv',index=False)

# 预测测试集label然后再和训练集一起训练再预测测试集（一轮）

In [181]:
X = a[a.y>=0].drop(['cust_id','y'],axis=1)
y = a[a.y>=0].y
lgb_data = lgb.Dataset(X, y)
res = lgb.cv(
    params,
    lgb_data,
    categorical_feature = cate_features2,
    num_boost_round=2000,
    nfold=5,
    stratified=False,
    metrics=None,
    early_stopping_rounds=150,
    verbose_eval=50,
    show_stdv=True,
    seed=0
)

In [182]:
iter_best = len(res['auc-mean'])
print(iter_best)

1671


In [183]:
res_train = lgb.train(
    params,
    lgb_data,
    categorical_feature = cate_features2,
    num_boost_round=1001
)

In [184]:
pred = res_train.predict(tmp3[tmp3.y==-2].drop(['cust_id','y'],axis=1))

In [811]:
result = tmp3[tmp3.y==-2][['cust_id']]
result['pred_prob'] = pred
result['cust_id'] = result['cust_id'].astype(int)

In [1000]:
result.to_csv('output/baseline9.csv',index=False)

# 将连续型的几个进行分箱 和 类别型一起onehot然后跑下LR看下效果

In [1029]:
a = tmp3[['cust_id','cust_group','y']+['x_1','x_2','x_80','x_81','x_93','x_95']+list(tmp3.iloc[:,87:112].columns)].copy()

In [1030]:
a.head()

,cust_id,cust_group,y,x_1,x_2,x_80,x_81,x_93,x_95,x_96,...,x_148,x_149,x_150,x_151,x_152,x_153,x_154,x_155,x_156,x_157
0,110000,3.0,0,0.354167,0.604988,0.865023,0.969147,0.208333,0.436170,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,-99.0
1,110001,3.0,0,0.125000,0.012058,0.820423,0.905626,0.270833,0.297872,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
2,110002,3.0,0,0.333333,0.565979,0.742958,0.867514,0.185821,0.372340,1.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
3,110003,3.0,0,0.208333,0.316209,0.761737,0.907441,0.010417,0.053191,1.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0
4,110004,3.0,0,0.208333,0.008061,0.926056,0.865699,0.052083,0.095745,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0


In [889]:
a[['y','x_1','x_2','x_80','x_81','x_93','x_95']].shape

(35000, 7)

In [1031]:
bb = Binning(a[['y','x_1','x_2','x_80','x_81','x_93','x_95']].copy(),label='y')
a_ = bb.binning()

Binning ...: 100%|██████████| 6/6 [00:00<00:00, 382.70it/s]


In [1032]:
a[['x_1','x_2','x_80','x_81','x_93','x_95']] = a_[['x_1','x_2','x_80','x_81','x_93','x_95']]

In [1033]:
a.head()

,cust_id,cust_group,y,x_1,x_2,x_80,x_81,x_93,x_95,x_96,...,x_148,x_149,x_150,x_151,x_152,x_153,x_154,x_155,x_156,x_157
0,110000,3.0,0,3,17,18,14,1,13,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,-99.0
1,110001,3.0,0,1,0,3,11,4,6,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
2,110002,3.0,0,3,5,1,7,2,6,1.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
3,110003,3.0,0,1,8,13,3,7,17,1.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0
4,110004,3.0,0,1,0,18,12,7,17,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0


In [1077]:
def get_onehot(df:pd.DataFrame, ):
    '''
    传入dataframe
    '''
    first_col_name = df.columns[0]
    tmp = pd.get_dummies(df[first_col_name])
    tmp.columns = [first_col_name+'_oht_0_'+str(x) for x in range(tmp.shape[1])]
    pd_sub = tmp.copy()
    for idx, i in enumerate(df.columns):
        if idx == 0:
            continue
        tmp = pd.get_dummies(df[i])
        tmp.columns = [first_col_name+'_oht_{}_'.format(str(idx))+str(x) for x in range(tmp.shape[1])]
        pd_sub = pd_sub.join(tmp.copy())
#     return pd.DataFrame(np_sub,columns=colnms)
    return pd_sub

In [1035]:
a_ = a[['y']].join(get_onehot(a.drop(['cust_id','y'],axis=1)))

In [1036]:
a_.head()

,y,cust_group_oht_0_0,cust_group_oht_0_1,cust_group_oht_0_2,cust_group_oht_1_0,cust_group_oht_1_1,cust_group_oht_1_2,cust_group_oht_1_3,cust_group_oht_1_4,cust_group_oht_1_5,...,cust_group_oht_30_1,cust_group_oht_30_2,cust_group_oht_30_3,cust_group_oht_31_0,cust_group_oht_31_1,cust_group_oht_31_2,cust_group_oht_31_3,cust_group_oht_31_4,cust_group_oht_31_5,cust_group_oht_31_6
0,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
1,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


# 用LR跑一下cv

In [1189]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier

In [1223]:
lr = LogisticRegression(C=3)
# lr = ExtraTreesClassifier(n_estimators=100,min_samples_leaf=5,max_depth=5,n_jobs=30,min_samples_split=5)
# lr = AdaBoostClassifier(n_estimators=100,learning_rate=0.07)
# lr = RandomForestClassifier(n_estimators=100,max_depth=5,min_samples_leaf=5,n_jobs=30)
X = tmp3[tmp3.y>=0].drop(['cust_id','y'],axis=1).copy(deep=True)
y = tmp3[tmp3.y>=0].y.copy(deep=True)

lr.fit(X, y)
pred = lr.predict_proba(tmp3[tmp3.y==-2].drop(['cust_id','y'],axis=1))[:,1]
print(pred.min(),pred.max())

0.007518343929539999 0.2070993116570183


In [1216]:
# kf = KFold(5, shuffle=True)
# for train_idx, val_idx in tqdm_notebook(kf.split(X)):
#     lr.fit(X.iloc[train_idx], y.iloc[train_idx])
#     print(roc_auc_score(y.iloc[val_idx], lr.predict_proba(X.iloc[val_idx])[:,-1])) # lr, etc, ada, gbc

0.7979728070371279
0.7947339153201237
0.8027311657903385
0.8030353422609149
0.8190997375813128


In [1224]:
result = tmp3[tmp3.y==-2][['cust_id']]
result['pred_prob'] = pred
result.to_csv('model_res/res_etc2.csv',index=False)